In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

%matplotlib inline


#You should download this from the site and change to your directory. Files too big for git to hold.
#https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9
df= pd.read_csv('C:/Users/Garland/Desktop/311_Service_Requests_from_2010_to_Present.csv', index_col=0)

df.head(5)

C:\Users\Garland\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,17,31,32,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
Unique Key,,,,,,,,,,,,,,,,,,,,,
23509318,06/27/2012 12:00:00 AM,07/02/2012 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,NONCONST,RUBBISH,RESIDENTIAL BUILDING,11221.0,1209 BUSHWICK AVENUE,BUSHWICK AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.688141,-73.915739,"(40.68814105386948, -73.91573940478034)"
23509319,06/27/2012 12:00:00 AM,07/05/2012 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,NONCONST,RUBBISH,RESIDENTIAL BUILDING,11221.0,71 STUYVESANT AVENUE,STUYVESANT AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.693516,-73.934153,"(40.693516344264985, -73.93415258394502)"
23509320,06/27/2012 09:02:26 AM,06/28/2012 12:00:00 AM,DOB,Department of Buildings,General Construction/Plumbing,Debris - Falling Or In Danger Of Falling,NaN,11357.0,154-33 11 AVENUE,11 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.792275,-73.806784,"(40.79227480990045, -73.80678362193075)"
23509321,06/27/2012 05:56:20 PM,06/29/2012 12:00:00 AM,DOB,Department of Buildings,General Construction/Plumbing,Debris - Falling Or In Danger Of Falling,NaN,10003.0,123 3 AVENUE,3 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.732979,-73.986848,"(40.732979398765266, -73.9868479762699)"
23509322,06/27/2012 11:12:22 AM,06/28/2012 12:00:00 AM,DOB,Department of Buildings,General Construction/Plumbing,Illegal Conversion Of Commercial Bldg/Space To...,NaN,11222.0,239 BANKER STREET,BANKER STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.726162,-73.956161,"(40.72616206569638, -73.95616079615964)"


In [2]:
df.shape

(2576231, 40)

In [3]:
df.columns

Index(['Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [4]:
df['Incident Zip'].unique()

array([11221., 11357., 10003., 11222., 10465., 10456., 11237., 11436.,
       11691., 11228., 11206., 11218., 10310., 10306., 11210., 11223.,
       11214., 11230., 10013., 11375., 11434., 11428., 11225., 11229.,
       11355., 10033., 10458., 11211., 10469., 10028., 11236., 11238.,
       10023., 10037., 10302., 10004., 11432., 11106., 11412., 10453.,
       10011., 10030., 11233., 10031., 11207., 10460., 11423., 10040.,
       10467., 11235., 10026., 11385., 11369., 10027., 11212., 10032.,
       11220., 10462., 11213., 11208., 11217., 11694., 11413., 10009.,
       11224., 11373., 11377., 10472., 11103., 10468., 10303., 11435.,
       10010., 10036., 11354., 10455., 11226., 10308., 10452., 10454.,
       11420., 11231., 11427., 10312., 11414., 11203., 11102., 10025.,
       11216., 10457., 10463., 10474., 10022., 10018., 10012., 11374.,
       11209., 10301., 10304., 11368., 11205., 11433., 11411., 10305.,
       11415., 11356., 10451., 10314., 11367., 11364., 11204., 10029.,
      

In [5]:
# Count null zips
print(df['Incident Zip'].isnull().value_counts())

False    2575356
True         875
Name: Incident Zip, dtype: int64


In [6]:
# Fill them with 0s and we will get rid of them

df['Incident Zip'].fillna(0.0, inplace = True)

In [7]:
print(df['Incident Zip'].isnull().value_counts())

False    2576231
Name: Incident Zip, dtype: int64


In [8]:
# Drop zip code errors

df = df.loc[~df['Incident Zip'].isin(['0.', '83.'])]

In [9]:
df['Incident Zip'].unique()

array([11221., 11357., 10003., 11222., 10465., 10456., 11237., 11436.,
       11691., 11228., 11206., 11218., 10310., 10306., 11210., 11223.,
       11214., 11230., 10013., 11375., 11434., 11428., 11225., 11229.,
       11355., 10033., 10458., 11211., 10469., 10028., 11236., 11238.,
       10023., 10037., 10302., 10004., 11432., 11106., 11412., 10453.,
       10011., 10030., 11233., 10031., 11207., 10460., 11423., 10040.,
       10467., 11235., 10026., 11385., 11369., 10027., 11212., 10032.,
       11220., 10462., 11213., 11208., 11217., 11694., 11413., 10009.,
       11224., 11373., 11377., 10472., 11103., 10468., 10303., 11435.,
       10010., 10036., 11354., 10455., 11226., 10308., 10452., 10454.,
       11420., 11231., 11427., 10312., 11414., 11203., 11102., 10025.,
       11216., 10457., 10463., 10474., 10022., 10018., 10012., 11374.,
       11209., 10301., 10304., 11368., 11205., 11433., 11411., 10305.,
       11415., 11356., 10451., 10314., 11367., 11364., 11204., 10029.,
      

In [10]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
submit2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test3.csv', index_col=0)

In [11]:
df.loc[df['Complaint Type'] == 'Radioactive Material']

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
Unique Key,,,,,,,,,,,,,,,,,,,,,
25283141,04/01/2013 02:52:07 PM,04/03/2013 08:58:24 AM,DOHMH,Department of Health and Mental Hygiene,Radioactive Material,Contamination Risk,Other (Explain Below),10009.0,239 EAST 7 STREET,EAST 7 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.723857,-73.978299,"(40.72385728070956, -73.97829942528917)"
26075251,08/06/2013 11:39:50 AM,08/09/2013 09:17:58 AM,DOHMH,Department of Health and Mental Hygiene,Radioactive Material,Material Stored Improperly,Hospital,11434.0,178-06 LINDEN BOULEVARD,LINDEN BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.692624,-73.769793,"(40.69262423408341, -73.76979291391517)"


In [12]:
df['Complaint Type'].unique()

array(['NONCONST', 'General Construction/Plumbing', 'Plumbing',
       'Sanitation Condition', 'Rodent', 'Street Condition',
       'Consumer Complaint', 'Street Light Condition',
       'Derelict Vehicles', 'Graffiti', 'Litter Basket / Request',
       'Recycling Enforcement', 'Special Projects Inspection Team (SPIT)',
       'Dirty Conditions', 'Missed Collection (All Materials)',
       'Traffic Signal Condition', 'BEST/Site Safety', 'PAINT - PLASTER',
       'PLUMBING', 'ELECTRIC', 'GENERAL CONSTRUCTION', 'HEATING',
       'Other Enforcement', 'Noise - Street/Sidewalk', 'Water System',
       'Unsanitary Animal Pvt Property', 'Air Quality', 'Noise', 'Sewer',
       'Noise Survey', 'Bus Stop Shelter Complaint', 'Building/Use',
       'Electrical', 'Elevator', 'Investigations and Discipline (IAD)',
       'Scaffold Safety', 'Hazardous Materials', 'Lead',
       'Water Conservation', 'APPLIANCE', 'Root/Sewer/Sidewalk Condition',
       'Overflowing Litter Baskets', 'Overgrown Tree/Bra

In [13]:
# Getting complaints that may sugguest a not very nice neighborhood, which can lower prices on some areas.
# Some complaints may help us determine the housing utilities as well.

new_df = df.loc[df['Complaint Type'].isin(['Sanitation Condition', 'Rodent', 'Street Condition', 'Dirty Conditions',
                                          'Noise - Street/Sidewalk', 'Noise', 'Root/Sewer/Sidewalk Condition', 'Asbestos',
                                          'Hazardous Materials', 'Unsanitary Pigeon Condition', 'Noise - Residential',
                                          'Drinking', 'Illegal Fireworks', 'Disorderly Youth', 'Homeless Encampment',
                                          'Poison Ivy', 'Building Condition', 'UNSANITARY CONDITION',
                                          'Noise - Commericial', 'Facades', 'Graffiti', 'PLUMBING', 'HEATING', 'ELECTRIC',
                                          'Boilers', 'Disorderly Youth'])]

new_df.head(5)

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
Unique Key,,,,,,,,,,,,,,,,,,,,,
23509339,06/27/2012 02:31:00 AM,06/27/2012 12:00:00 PM,DSNY,Queens East 12,Sanitation Condition,12 Dead Animals,Sidewalk,11436.0,116 AVENUE,116 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.679962,-73.802209,"(40.679962465998194, -73.80220932208167)"
23509340,06/27/2012 03:25:00 PM,06/28/2012 12:00:00 PM,DSNY,BCC - Queens East,Sanitation Condition,22 Weeds,Sidewalk,11691.0,330 BEACH 40 STREET,BEACH 40 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.595054,-73.772160,"(40.59505427220381, -73.772160146091)"
23509344,06/27/2012 12:00:00 AM,07/31/2012 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Apt. Building,11228.0,901 80 STREET,80 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.622683,-74.019020,"(40.62268259782637, -74.0190200085561)"
23509345,06/27/2012 12:00:00 AM,06/27/2012 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Apt. Building,11206.0,128 MANHATTAN AVENUE,MANHATTAN AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706936,-73.944859,"(40.70693564721781, -73.9448588214993)"
23509346,06/27/2012 12:00:00 AM,07/16/2012 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Construction Site,11218.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.647693,-73.980364,"(40.647692997464205, -73.98036384915527)"


In [14]:
new_df.shape

(1158574, 40)

In [15]:
#Filtering by date. 
#Thinking we should check the previous years as some neighborhood "rep" could still be in effect even if it was stated resolved. 

#new_df['Created Date'] = pd.to_datetime(new_df['Created Date'], format = '%m/%d/%Y %H:%M:%S %p')

new_df[(new_df['Created Date'] > '01/01/2010 12:00:00 AM') & (new_df['Created Date'] < '08/01/2013 12:00:00 AM')]

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
Unique Key,,,,,,,,,,,,,,,,,,,,,
23509339,06/27/2012 02:31:00 AM,06/27/2012 12:00:00 PM,DSNY,Queens East 12,Sanitation Condition,12 Dead Animals,Sidewalk,11436.0,116 AVENUE,116 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.679962,-73.802209,"(40.679962465998194, -73.80220932208167)"
23509340,06/27/2012 03:25:00 PM,06/28/2012 12:00:00 PM,DSNY,BCC - Queens East,Sanitation Condition,22 Weeds,Sidewalk,11691.0,330 BEACH 40 STREET,BEACH 40 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.595054,-73.772160,"(40.59505427220381, -73.772160146091)"
23509344,06/27/2012 12:00:00 AM,07/31/2012 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Apt. Building,11228.0,901 80 STREET,80 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.622683,-74.019020,"(40.62268259782637, -74.0190200085561)"
23509345,06/27/2012 12:00:00 AM,06/27/2012 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Apt. Building,11206.0,128 MANHATTAN AVENUE,MANHATTAN AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.706936,-73.944859,"(40.70693564721781, -73.9448588214993)"
23509346,06/27/2012 12:00:00 AM,07/16/2012 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Construction Site,11218.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.647693,-73.980364,"(40.647692997464205, -73.98036384915527)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28606886,07/31/2014 08:58:20 AM,08/01/2014 06:15:00 PM,DOT,Department of Transportation,Street Condition,Pothole,NaN,10016.0,40 PARK AVENUE,PARK AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.748549,-73.980284,"(40.74854943630217, -73.98028359436955)"
28607440,07/31/2014 09:00:50 AM,08/01/2014 07:40:00 PM,DOT,Department of Transportation,Street Condition,Pothole,NaN,10007.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.713135,-74.004058,"(40.71313547814755, -74.00405805807038)"
28607963,07/29/2014 09:32:00 AM,08/05/2014 12:00:00 PM,DSNY,A - Bronx,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,10460.0,1796 VYSE AVENUE,VYSE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.837583,-73.885472,"(40.837582696561796, -73.88547154114725)"


In [16]:
new_df['Incident Zip'].unique()

array([11436., 11691., 11228., 11206., 11218., 10310., 10306., 11210.,
       11223., 11214., 11230., 11225., 11229., 11355., 10033., 11434.,
       11236., 10023., 10037., 11106., 11412., 11222., 10011., 10027.,
       10458., 11212., 10453., 10032., 11207., 11220., 10467., 10462.,
       11213., 11208., 10030., 11694., 11413., 10009., 11224., 10003.,
       11373., 11377., 10472., 11428., 11103., 10302., 10452., 10040.,
       10028., 10454., 11375., 11420., 11233., 11231., 11427., 10312.,
       10468., 10457., 11217., 10022., 10018., 11374., 10010., 10029.,
       11358., 11361., 11219., 10024., 10456., 11221., 11226., 10470.,
       11421., 10002., 10025., 10471., 11215., 11385., 10026., 11216.,
       10461., 11418., 10304., 11238., 11004., 10463., 10466., 11416.,
       10314., 11249., 11235., 11369., 11368., 11211., 11234., 11419.,
       10035., 11372., 11414., 10303., 10465., 11209., 11354., 11415.,
       11101., 11432., 10459., 10460., 10016., 10019., 10021., 10308.,
      

In [17]:
new_df['Incident Zip'].unique()

array([11436., 11691., 11228., 11206., 11218., 10310., 10306., 11210.,
       11223., 11214., 11230., 11225., 11229., 11355., 10033., 11434.,
       11236., 10023., 10037., 11106., 11412., 11222., 10011., 10027.,
       10458., 11212., 10453., 10032., 11207., 11220., 10467., 10462.,
       11213., 11208., 10030., 11694., 11413., 10009., 11224., 10003.,
       11373., 11377., 10472., 11428., 11103., 10302., 10452., 10040.,
       10028., 10454., 11375., 11420., 11233., 11231., 11427., 10312.,
       10468., 10457., 11217., 10022., 10018., 11374., 10010., 10029.,
       11358., 11361., 11219., 10024., 10456., 11221., 11226., 10470.,
       11421., 10002., 10025., 10471., 11215., 11385., 10026., 11216.,
       10461., 11418., 10304., 11238., 11004., 10463., 10466., 11416.,
       10314., 11249., 11235., 11369., 11368., 11211., 11234., 11419.,
       10035., 11372., 11414., 10303., 10465., 11209., 11354., 11415.,
       11101., 11432., 10459., 10460., 10016., 10019., 10021., 10308.,
      

In [18]:
# Summing up the number of times each zip appeared

new_df['Incident Zip'].value_counts()

11226.0    28126
10467.0    26161
10458.0    22856
10468.0    21537
10453.0    20407
           ...  
10103.0        4
10153.0        3
11241.0        3
11359.0        3
11242.0        2
Name: Incident Zip, Length: 201, dtype: int64

In [19]:
#new_df['Incident Zip'] == '11226'

new_df['Incident Zip'].dtype

dtype('float64')

In [20]:
new_df['Incident Zip'] = new_df['Incident Zip'].astype('int64')

C:\Users\Garland\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
new_df.tail(5)

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
Unique Key,,,,,,,,,,,,,,,,,,,,,
28750551,08/25/2014 12:00:00 AM,09/10/2014 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,ELECTRIC,OUTLET/SWITCH,RESIDENTIAL BUILDING,11236,9320 AVENUE L,AVENUE L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.636865,-73.898084,"(40.636865247686245, -73.89808438974755)"
28750553,08/25/2014 12:37:00 PM,08/25/2014 12:37:00 PM,DSNY,BCC - Queens West,Sanitation Condition,12 Dead Animals,Sidewalk,11105,23-79 24 STREET,24 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.775940,-73.919170,"(40.775940009617706, -73.91917027518858)"
28750565,08/25/2014 12:00:00 AM,09/19/2014 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,ELECTRIC,WIRING,RESIDENTIAL BUILDING,11226,2608 ALBEMARLE ROAD,ALBEMARLE ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.648003,-73.952410,"(40.6480032224184, -73.95241030934798)"
28750571,08/25/2014 12:00:00 AM,09/07/2014 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,ELECTRIC,WIRING,RESIDENTIAL BUILDING,11691,18-26 EVERDELL AVENUE,EVERDELL AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.600933,-73.750912,"(40.60093347617232, -73.75091151513281)"
28750590,08/25/2014 12:00:00 AM,09/12/2014 12:00:00 AM,HPD,Department of Housing Preservation and Develop...,PLUMBING,BASIN/SINK,RESIDENTIAL BUILDING,11421,86-06 88 AVENUE,88 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.690858,-73.857345,"(40.69085844658601, -73.85734464996217)"


In [28]:
#Count up the total complaints by zip code
zip_incident_count = new_df['Incident Zip'].value_counts()
zip_incident_count

11226    28126
10467    26161
10458    22856
10468    21537
10453    20407
         ...  
10103        4
11359        3
10153        3
11241        3
11242        2
Name: Incident Zip, Length: 201, dtype: int64

In [29]:
#Making a df with what we got before.
zip_incident_df = zip_incident_count.rename_axis('Incident Zip Code').reset_index(name = 'Number of Incidents')
print(zip_incident_df)

     Incident Zip Code  Number of Incidents
0                11226                28126
1                10467                26161
2                10458                22856
3                10468                21537
4                10453                20407
..                 ...                  ...
196              10103                    4
197              11359                    3
198              10153                    3
199              11241                    3
200              11242                    2

[201 rows x 2 columns]


In [31]:
train_df.columns

Index(['addr_unit', 'building_id', 'bedrooms', 'bathrooms', 'size_sqft',
       'created_at', 'addr_street', 'addr_city', 'addr_zip', 'addr_lat',
       'addr_lon', 'bin', 'bbl', 'floor_count', 'year_built', 'min_to_subway',
       'has_doorman', 'has_elevator', 'has_fireplace', 'has_dishwasher',
       'is_furnished', 'has_gym', 'allows_pets', 'has_washer_dryer',
       'has_garage', 'has_roofdeck', 'has_concierge', 'has_pool', 'has_garden',
       'has_childrens_playroom', 'rent', 'no_fee', 'description',
       'neighborhood', 'borough', 'unit', 'floornumber', 'line'],
      dtype='object')

In [32]:
test_df.columns

Index(['addr_unit', 'building_id', 'bedrooms', 'bathrooms', 'size_sqft',
       'created_at', 'addr_street', 'addr_city', 'addr_zip', 'addr_lat',
       'addr_lon', 'bin', 'bbl', 'floor_count', 'year_built', 'min_to_subway',
       'has_doorman', 'has_elevator', 'has_fireplace', 'has_dishwasher',
       'is_furnished', 'has_gym', 'allows_pets', 'has_washer_dryer',
       'has_garage', 'has_roofdeck', 'has_concierge', 'has_pool', 'has_garden',
       'has_childrens_playroom', 'rent', 'no_fee', 'description',
       'neighborhood', 'borough', 'unit', 'floornumber', 'line'],
      dtype='object')

In [35]:
#Merge with train and test data
merged_train = pd.merge(train_df, zip_incident_df, how='outer', on=['addr_zip','Incident Zip Code'])
merged_test = pd.merge(test_df, zip_incident_df,how='outer', on=['addr_zip','Incident Zip Code'])

KeyError: 'addr_zip'